## Replication: When to use and when not to use BBR: An empirical analysis and evaluation study

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="re-when-to-use-bbr-" + fablib.get_bastion_username()

# use default_ubuntu_22 for 5.15 kernel, default_ubuntu_18 for 4.15 kernel, default_ubuntu_20 for BBRv2
image = 'default_ubuntu_22' 

node_conf = [
 {'name': "h1",  'cores': 4, 'ram': 16, 'disk': 500, 'image': image, 'packages': ['iperf3']}, 
 {'name': "h2",  'cores': 4, 'ram': 16, 'disk': 10, 'image': image, 'packages': ['iperf3']}, 
 {'name': "h3",  'cores': 4, 'ram': 16, 'disk': 10, 'image': image, 'packages': ['iperf3']}, 
 {'name': "tbf", 'cores': 4, 'ram': 16, 'disk': 10, 'image': image, 'packages': []} 
]
net_conf = [
 {"name": "link1", "subnet": "10.10.1.0/24", "nodes": [{"name": "tbf",   "addr": "10.10.1.10"}, {"name": "h1", "addr": "10.10.1.1"}]},
 {"name": "link2", "subnet": "10.10.2.0/24", "nodes": [{"name": "tbf",   "addr": "10.10.2.10"}, {"name": "h2", "addr": "10.10.2.1"}]},
 {"name": "link3", "subnet": "10.10.3.0/24", "nodes": [{"name": "tbf",   "addr": "10.10.3.10"}, {"name": "h3", "addr": "10.10.3.1"}]}
]
route_conf = [
 {"addr": "10.10.3.0/24", "gw": "10.10.1.10", "nodes": ["h1"]}, 
 {"addr": "10.10.3.0/24", "gw": "10.10.2.10", "nodes": ["h2"]},  
 {"addr": "10.10.1.0/24", "gw": "10.10.3.10", "nodes": ["h3"]},  
 {"addr": "10.10.2.0/24", "gw": "10.10.3.10", "nodes": ["h3"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

### Extra configuration for this experiment

In [ ]:
# set socket read and write buffer on all endpoints to larger value
for node in slice.get_nodes():
  node.execute("sudo sysctl -w net.core.rmem_default=2147483647")
  node.execute("sudo sysctl -w net.core.wmem_default=2147483647")
  node.execute("sudo sysctl -w net.core.rmem_max=2147483647")
  node.execute("sudo sysctl -w net.core.wmem_max=2147483647")

### Validate base network

Before we run any underlying experiment, we should check the “base” network - before adding any emulated delay or rate limiting - and make sure that it will not be a limiting factor in the experiment.

In [ ]:
# check base delay
slice.get_node("h1").execute("ping -c 5 h3")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
slice.get_node("h3").execute("iperf3 -s -1 -D")
time.sleep(5)
slice.get_node("h1").execute("iperf3 -t 30 -i 10 -P 10 -c h3")

In [ ]:
# also check Linux kernel version on sender
slice.get_node("h1").execute("uname -a")

#### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Execute experiment

In [ ]:
# get name of router interface that is 'toward' h1 - apply delay here
# get name of router interface that is 'toward' h3 - apply rate limiting and buffer size limit here
router_node = slice.get_node(name='tbf')
router_ingress_iface = router_node.get_interface(network_name = "link1")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "link3")
router_egress_name = router_egress_iface.get_device_name()

In [ ]:
tx_node = slice.get_node(name="h1")
rx_node = slice.get_node(name="h3")

tx_node.execute("mkdir -p fig5")

In [ ]:
for bufcap in [100, 10000]:
    for bandwidth in [10, 20, 50, 100, 250, 500, 750, 1000]: 
        for rtt in [5, 10, 25, 50, 75, 100, 150, 200]: 

                # set up RTT
        router_node.execute("sudo tc qdisc replace dev " + router_ingress_name + " root netem delay " + str(rtt) + "ms")
                # set up rate limit, buffer limit
                router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3")
                router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate " + str(bandwidth) + "Mbit")
                router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " parent 1:3 bfifo limit " + str(bufcap) + "kb")
        
        # quick validation
        tx_node.execute("ping -c 5 h3")
        rx_node.execute("iperf3 -s -1 -D")
        time.sleep(5)
        tx_node.execute("iperf3 -t 30 -i 30 -P 10 -c h3")
        time.sleep(10)

        for trial_idx in [1, 2, 3, 4, 5]:

                        file_prefix = "fig5/%d_%d_%d_%d_" % (bufcap, bandwidth, rtt, trial_idx)

            # cubic experiment
            time.sleep(10)
            rx_node.execute("iperf3 -s -1 -D")
            tx_node.execute("iperf3 -c h3 -C cubic -t 60s -fk > fig5/" + file_prefix + "_cubic.txt")

                        # bbr experiment
                        time.sleep(10)
                        rx_node.execute("iperf3 -s -1 -D")
                        tx_node.execute("iperf3 -c h3 -C bbr -t 60s -fk > fig5/" + file_prefix + "_bbr.txt")


### Analyze experiment results

### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()